# Overview

This project aims to create a trading bot that implements a high frequency trading strategy. Currently it trades SPY and uses a variety of technical indicators such as rolling averages, RSI, MACD, and Bollinger Bands. The bot only trades during market hours due to liquidity. I'm going to continue to test different strategies and see which ones generate the greatest return. I'm also going to test which stocks are better for the bot. To start off, I chose the SPY ETF becuase of its high implied volatility. I predict that the bot will generate greater returns by trading securities that make bigger moves on a day to day basis.

# Install Necessary Packages

In [17]:
pip install pandas numpy pandas_ta alpaca-trade-api

# Import Libraries and Initialize API

In [18]:
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
import pandas_ta as ta
import time

# Initialize Alpaca API
api = tradeapi.REST('PKVBLESV0BDW3WA084CL', 'Bxrojd5FvQApXh0QK25bo1phmTh9nh7he1pC4eF1', base_url='https://paper-api.alpaca.markets')

# Check Market Status

In [24]:
# Function to check if the market is open
def is_market_open():
    clock = api.get_clock()
    return clock.is_open

# Function to wait until the market opens
def wait_for_market_open():
    while not is_market_open():
        print("Market is closed. Waiting for market to open...")
        time.sleep(60)

# Fetch Historical Data

In [25]:
# Function to fetch historical data for SPY
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    try:
        barset = api.get_bars(symbol, timeframe, start=start_date, end=end_date)
        data = barset.df
        if data is None or data.empty:
            print(f"No data fetched for {symbol} from {start_date} to {end_date}")
            return None
        data.columns = [col.split('_')[0] for col in data.columns]  # Ensure single-level column index
        return data
    except tradeapi.rest.APIError as e:
        print(f"APIError: {e}")
        return None

# Fetch historical 1-minute data for SPY
symbol = "SPY"
timeframe = "1Min"
start_date = "2023-05-01"
end_date = "2023-05-31"
data = fetch_historical_data(symbol, timeframe, start_date, end_date)

if data is not None:
    print(data.head())
else:
    print("Failed to fetch historical data.")

                            close    high     low  trade    open  volume  \
timestamp                                                                  
2023-05-01 08:00:00+00:00  416.07  416.15  416.07     86  416.15   52935   
2023-05-01 08:01:00+00:00  416.07  416.07  416.07     30  416.07   22205   
2023-05-01 08:02:00+00:00  416.14  416.15  416.14     30  416.15   25508   
2023-05-01 08:04:00+00:00  416.12  416.12  416.12      6  416.12    1001   
2023-05-01 08:05:00+00:00  416.11  416.11  416.10     23  416.10   18856   

                                 vwap  
timestamp                              
2023-05-01 08:00:00+00:00  416.121872  
2023-05-01 08:01:00+00:00  416.069999  
2023-05-01 08:02:00+00:00  416.149907  
2023-05-01 08:04:00+00:00  416.119860  
2023-05-01 08:05:00+00:00  416.109917  


# Calculate Technical Indicators

In [26]:
if data is not None:
    # Define rolling windows
    short_window = 5  # 5-minute rolling average
    long_window = 20  # 20-minute rolling average

    # Calculate rolling averages
    data['short_ma'] = data['close'].rolling(window=short_window).mean()
    data['long_ma'] = data['close'].rolling(window=long_window).mean()

    # Calculate RSI
    data['rsi'] = ta.rsi(data['close'], length=14)

    # Calculate MACD
    macd = ta.macd(data['close'], fast=12, slow=26, signal=9)
    data['macd'] = macd['MACD_12_26_9']
    data['macd_signal'] = macd['MACDs_12_26_9']
    data['macd_hist'] = macd['MACDh_12_26_9']

    # Calculate Bollinger Bands
    bollinger = ta.bbands(data['close'], length=20, std=2)
    data['upper_band'] = bollinger['BBU_20_2.0']
    data['middle_band'] = bollinger['BBM_20_2.0']
    data['lower_band'] = bollinger['BBL_20_2.0']

    # Generate signals
    data['signal'] = 0
    data['signal'][short_window:] = np.where((data['short_ma'][short_window:] > data['long_ma'][short_window:]) &
                                             (data['rsi'][short_window:] < 70) &
                                             (data['macd_hist'][short_window:] > 0) &
                                             (data['close'][short_window:] < data['upper_band'][short_window:]), 1, 0)
    data['signal'][short_window:] = np.where((data['short_ma'][short_window:] < data['long_ma'][short_window:]) &
                                             (data['rsi'][short_window:] > 30) &
                                             (data['macd_hist'][short_window:] < 0) &
                                             (data['close'][short_window:] > data['lower_band'][short_window:]), -1, data['signal'][short_window:])

    data['position'] = data['signal'].diff()

    # Drop rows with NaN values
    data.dropna(inplace=True)

    # Check the structure of the data with signals
    print(data.head())

                            close    high     low  trade    open  volume  \
timestamp                                                                  
2023-05-01 08:47:00+00:00  415.79  415.79  415.79      5  415.79     195   
2023-05-01 08:48:00+00:00  415.78  415.78  415.78      3  415.78     193   
2023-05-01 08:49:00+00:00  415.78  415.78  415.78      4  415.78     219   
2023-05-01 08:53:00+00:00  415.75  415.75  415.75      2  415.75     302   
2023-05-01 08:54:00+00:00  415.78  415.78  415.78      2  415.78     135   

                                 vwap  short_ma   long_ma        rsi  \
timestamp                                                              
2023-05-01 08:47:00+00:00  415.790667   415.722  415.7105  52.149064   
2023-05-01 08:48:00+00:00  415.779948   415.746  415.7145  51.014289   
2023-05-01 08:49:00+00:00  415.788311   415.762  415.7200  51.014289   
2023-05-01 08:53:00+00:00  415.749868   415.770  415.7215  47.423826   
2023-05-01 08:54:00+00:00  415.7981

<ipython-input-26-190be6d39334>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['signal'][short_window:] = np.where((data['short_ma'][short_window:] > data['long_ma'][short_window:]) &
<ipython-input-26-190be6d39334>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['signal'][short_window:] = np.where((data['short_ma'][short_window:] < data['long_ma'][short_window:]) &


# Backtest the Strategy

In [27]:
if data is not None:
    # Define initial capital and positions
    initial_capital = 100000.0
    positions = pd.DataFrame(index=data.index).fillna(0.0)
    positions[symbol] = data['signal']

    # Calculate portfolio values
    portfolio = positions.multiply(data['close'], axis=0)
    pos_diff = positions.diff()
    portfolio['holdings'] = (positions.multiply(data['close'], axis=0)).sum(axis=1)
    portfolio['cash'] = initial_capital - (pos_diff.multiply(data['close'], axis=0)).sum(axis=1).cumsum()
    portfolio['total'] = portfolio['cash'] + portfolio['holdings']

    # Print final portfolio value and ROI
    final_value = portfolio['total'].iloc[-1]
    print(f'Final portfolio value: ${final_value:.2f}')
    print(f'Return on investment: {((final_value / initial_capital) - 1) * 100:.2f}%')

Final portfolio value: $100402.68
Return on investment: 0.40%


# Live Trading Function

In [28]:
# Define risk management parameters
initial_cash = 100000.0
cash = initial_cash
shares = 0
stop_loss_pct = 0.01  # 1% stop-loss
take_profit_pct = 0.02  # 2% take-profit
max_position_size_pct = 0.1  # Max position size as a percentage of cash

# Function to execute trades
def execute_trade(symbol, current_price, signal, cash, shares):
    qty = int((cash * max_position_size_pct) // current_price)
    if signal == 1 and cash > current_price:  # Buy signal
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side='buy',
            type='market',
            time_in_force='gtc'
        )
        shares += qty
        cash -= qty * current_price
    elif signal == -1 and shares > 0:  # Sell signal
        api.submit_order(
            symbol=symbol,
            qty=shares,
            side='sell',
            type='market',
            time_in_force='gtc'
        )
        cash += shares * current_price
        shares = 0
    return cash, shares

# Live Trading

In [29]:
# Wait for the market to open before starting live trading
wait_for_market_open()

# Live trading with advanced risk management
while True:
    try:
        # Ensure the market is open
        if not is_market_open():
            print("Market is closed. Waiting for market to open...")
            time.sleep(60)
            continue

        # Fetch latest 1-minute data for SPY
        latest_data = api.get_bars(symbol, '1Min', limit=long_window + 1).df
        if latest_data is None or latest_data.empty:
            raise ValueError("No data fetched")

        latest_data.columns = [col.split('_')[0] for col in latest_data.columns]

        # Calculate rolling averages
        latest_data['short_ma'] = latest_data['close'].rolling(window=short_window).mean()
        latest_data['long_ma'] = latest_data['close'].rolling(window=long_window).mean()

        # Calculate RSI
        latest_data['rsi'] = ta.rsi(latest_data['close'], length=14)

        # Calculate MACD
        macd = ta.macd(latest_data['close'], fast=12, slow=26, signal=9)
        latest_data['macd'] = macd['MACD_12_26_9']
        latest_data['macd_signal'] = macd['MACDs_12_26_9']
        latest_data['macd_hist'] = macd['MACDh_12_26_9']

        # Calculate Bollinger Bands
        bollinger = ta.bbands(latest_data['close'], length=20, std=2)
        latest_data['upper_band'] = bollinger['BBU_20_2.0']
        latest_data['middle_band'] = bollinger['BBM_20_2.0']
        latest_data['lower_band'] = bollinger['BBL_20_2.0']

        # Check if the latest data has enough data points for indicators
        if latest_data.shape[0] < long_window + 1:
            raise ValueError("Insufficient data points")

        # Get the latest price and signals
        current_price = latest_data['close'].iloc[-1]
        short_ma = latest_data['short_ma'].iloc[-1]
        long_ma = latest_data['long_ma'].iloc[-1]
        rsi = latest_data['rsi'].iloc[-1]
        macd_hist = latest_data['macd_hist'].iloc[-1]
        upper_band = latest_data['upper_band'].iloc[-1]
        lower_band = latest_data['lower_band'].iloc[-1]

        # Debug information
        print(f"Current Price: {current_price}")
        print(f"Short MA: {short_ma}, Long MA: {long_ma}")
        print(f"RSI: {rsi}, MACD Hist: {macd_hist}")
        print(f"Upper Band: {upper_band}, Lower Band: {lower_band}")

        # Generate trading signal
        if (short_ma > long_ma) and (rsi < 70) and (macd_hist > 0) and (current_price < upper_band):
            signal = 1
        elif (short_ma < long_ma) and (rsi > 30) and (macd_hist < 0) and (current_price > lower_band):
            signal = -1
        else:
            signal = 0

        # Execute trade based on signal
        cash, shares = execute_trade(symbol, current_price, signal, cash, shares)

        # Risk management: stop-loss and take-profit
        if shares > 0:
            stop_loss_price = current_price * (1 - stop_loss_pct)
            take_profit_price = current_price * (1 + take_profit_pct)

            if current_price <= stop_loss_price or current_price >= take_profit_price:
                api.submit_order(
                    symbol=symbol,
                    qty=shares,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )
                cash += shares * current_price
                shares = 0

        # Wait for the next minute (ensures we stay well within the API limit)
        time.sleep(60)
    except tradeapi.rest.APIError as e:
        print(f"APIError: {e}")
        time.sleep(60)  # Wait before retrying
    except ValueError as e:
        print(f"ValueError: {e}")
        time.sleep(60)  # Wait before retrying
    except Exception as e:
        print(f"Unexpected error: {e}")
        time.sleep(60)  # Wait before retrying

Market is closed. Waiting for market to open...


KeyboardInterrupt: 